In [ ]:
import sys
sys.path.append('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main')
from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens100KReader import Movielens100KReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens100KReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data("/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/CiteULike_a/data") #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

In [ ]:
import optuna
import numpy as np
import pandas as pd
import os

def create_objective_d(recommender_instance, URM_train, URM_validation, hyperparam_space, evaluator_validation, cutoff, directory_path):
    def objective(trial):
        # Gather and set hyperparameters
        hyperparams = {name: trial.suggest_categorical(name, space) for name, space in hyperparam_space.items()}

        # Initialize and train the recommender
        
        recommender_instance.fit(**hyperparams)

        # Evaluate recommender
        user_id_array = np.arange(URM_validation.shape[0])
        
        result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
        result_df['hyperparams'] = str(hyperparams)

        # Assuming df_row is the DataFrame containing the row to append

        filename = directory_path + recommender_instance.RECOMMENDER_NAME + ".csv"

        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)
    
        return result_df.loc[cutoff]['NDCG']

    return objective


In [1]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

OSError: dlopen(/Users/lucaortolomo/opt/anaconda3/envs/TESI/lib/python3.8/site-packages/torch/lib/libtorch_global_deps.dylib, 0x000A): tried: '/Users/lucaortolomo/opt/anaconda3/envs/TESI/lib/python3.8/site-packages/torch/lib/libtorch_global_deps.dylib' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/lucaortolomo/opt/anaconda3/envs/TESI/lib/python3.8/site-packages/torch/lib/libtorch_global_deps.dylib' (no such file), '/Users/lucaortolomo/opt/anaconda3/envs/TESI/lib/python3.8/site-packages/torch/lib/libtorch_global_deps.dylib' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/Users/lucaortolomo/opt/anaconda3/envs/Tesi/lib/python3.8/site-packages/torch/lib/libtorch_global_deps.dylib' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/lucaortolomo/opt/anaconda3/envs/Tesi/lib/python3.8/site-packages/torch/l

In [ ]:
from Diffusion.DiffusionRecommender import SimpleAttentionDiffusionRecommender
diffusion_model = SimpleAttentionDiffusionRecommender(URM_train = URM_train, use_gpu = True)
diffusion_model.fit(epochs=50)